# this notebook is used to generate a tseries, with an SLM mask at different PV laser powers. This enables PMT gain calculations

In [1]:
import win32com.client
from datetime import date
import numpy as np
import time

In [2]:
# Start PrairieLink
pl = win32com.client.Dispatch('PrairieLink.Application')

# Connect to Client
pl.Connect()
print(pl.Connected)

com_error: (-2147221164, 'Class not registered', None, None)

# different laser powers -> different means -> plot variance vs mean -> get regression slope, interpret this as gain ( ASSUMPTION: poisson distributed tstack)
# different PMT voltages -> get different regression slopes -> plot gain vs PMT voltage

# set Pmin (laser power in PV) and Pmax manually, CHECK FIRST if they max out loookup table for EVERY PMT value

In [ ]:
# set modalities of the 2d-parameter-space scan
# do this manually!!
Pmin = 200
Pstep = 50

# PV that corresponds to 50mw OnSample
# maybe not
Pmax = 700

# Volate were we can see
Vmin = 345
Vstep = 5

# Voltage that slightly oversaturates the uniformly lit sample
Vmax = 355

# length of timestack that we will take the mean over
tStack_length = 101

# Laser name, wavelength and PLT number - please make sure that you can see
laser_name = "Imaging"
pmt_number = 2
wavelength = 765


# iterate through P and V to acquire tstacks
Vspace = np.arange(Vmin,Vmax+Vstep,Vstep)
Pspace = np.arange(Pmin,Pmax+Pstep,Pstep)

In [ ]:
# construct Packer-xkcd-compliant datestring
datestring = date.today().isoformat()

# update settings once
#pl.SendScriptCommands('-SetSavePath F:\Data\mloidolt\2018-07-20\MatLabFolder')
#pl.SendScriptCommands('-SetAcquisitionMode Galvo')
#pl.SendScriptCommands('-TSeriesLoad F:\Data\mloidolt\Configs\PMT-sensitivity-tstackConfig.env')
print('Folders set')

In [ ]:
for V in Vspace:
    for P in Pspace:
        pl.SendScriptCommands('-SetFileName ' + 'Tseries ' + datestring + 'V' + str(int(V)) + 'P' + str(int(P)))
        pl.SendScriptCommands('-SetLaserPower ' + laser_name + ' ' + str(P))
        pl.SendScriptCommands('-SetPMTGain ' + str(pmt_number) + ' ' + str(V))
        pl.SendScriptCommands('-TSeries')
        pl.SendScriptCommands('-WaitForScan')
        time.sleep(3)
        print(P)
    print(V)

pl.SendScriptCommands('-SetPMTGain ' + str(pmt_number) + ' ' + str(0))
pl.SendScriptCommands('-SetLaserPower ' + laser_name + ' ' + str(0))
#pl.Disconnect()

In [ ]:
pl.Disconnect()